# Exercício Aula 04 - Centralidade

Author: Gabriel Van Loon
Prof.:  Francisco Aparecido Rodrigues
Universidade de São Paulo, São Carlos, Brasil.


## Definindo as Funções e Bibliotecas utilizadas

In [27]:
import collections
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from scipy.stats import pearsonr, spearmanr

Funções vistas na aula do exercício da aula 04

In [ ]:
# Todas opções de centralidades são implementadas pelo networkx (muito eficiente u.u)

Funções vistas na aula do exercício da aula 03

In [26]:
def short_path_avg_var(G):
    dists = np.zeros(len(G)*(len(G)-1))
    paths = np.zeros((len(G),len(G)))    
    aux = 0
    for i in range(len(G)):
        print("Calculating row:", i)
        for j in range(i+1, len(G)):
            if i != j:
                dists[aux] = dists[(aux+1)] = len(nx.shortest_path(G,i,j))-1
                paths[i,j] = paths[j,i] = dists[aux]
                aux        += 2
                
    return np.average(dists), np.var(dists), paths

def pearson_degree_correlation(G):
    ki = []
    kj = []
    for i in range(0,len(G.nodes())):
        for j in range(0, len(G.nodes())):
            if(G.has_edge(i,j) == True):
                ki.append(G.degree(i))
                kj.append(G.degree(j))
    
    corr, _ = pearsonr(ki, kj)
    return corr, ki, kj

Funções vistas na aula do exercício da aula 02

In [3]:
def network_degree(G):
    vk = np.array([v for v in dict(G.degree()).values()])
    return vk, avg_degree

# Optimized version by networkx documentation
def degree_distribution(G, normalize=False):
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
    degree_count    = collections.Counter(degree_sequence)
    
    if normalize:
        total_sum = sum(degree_count.values())
        for degree in degree_count:
            degree_count[degree] = degree_count[degree]/total_sum
            
    return degree_count

# Francisco originals
def degree_distribution_original(G):
    vk = dict(G.degree())
    vk = list(vk.values())  # we get only the degree values
    vk = np.array(vk)
    maxk = np.max(vk)
    mink = np.min(vk)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk

def momment_of_degree_distribution(G,m):
    M = 0
    N = len(G)
    for i in G.nodes:
        M = M + G.degree(i)**m
    M = M/N
    return M

def network_variance(G):
    variance = momment_of_degree_distribution(G,2) - momment_of_degree_distribution(G,1)**2
    return variance

def shannon_entropy(G):
    k,Pk = degree_distribution_original(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H

def normalized_shannon_entropy(G):
    k,Pk = degree_distribution_original(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H/np.math.log(len(G),2)

def network_complexity_coef(G):
    return (momment_of_degree_distribution(G,2) / momment_of_degree_distribution(G,1))

# Just to remember the calls
# nx.transitivity(G)
# nx.average_clustering(G)

## 1) A media da medida eigenvector centrality da rede USairport500 é igual a quanto? Considere apenas o maior componente

In [8]:
# Read and prepare the network
G = nx.read_edgelist('data/USairport500.txt', nodetype=int)
# Grafo não direcionado
G = G.to_undirected()
# Remover auto-loops
G.remove_edges_from(nx.selfloop_edges(G))
# Escolhe maior componente
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
# Renomeia os vértices
G = nx.convert_node_labels_to_integers(G, first_label=0)

In [9]:
# Calculating the Eigenvector with numerical methods 
eigen_centrality    = dict(nx.eigenvector_centrality(G, max_iter = 1000))
avg_eigen_centrality = np.mean(list(eigen_centrality.values()))

print('Average eigenvector centrality', avg_eigen_centrality)
# print('Eigenvetor centrality:', EC[:10])

Average eigenvector centrality 0.022754398239423695


## 2) Calcule a Correlação de Pearson entre as medidas Betweeness e Grau Centrality para a rede Hamsterster

In [10]:
# Read and prepare the network
G = nx.read_edgelist('data/hamsterster.txt', nodetype=int)
# Grafo não direcionado
G = G.to_undirected()
# Remover auto-loops
G.remove_edges_from(nx.selfloop_edges(G))
# Escolhe maior componente
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
# Renomeia os vértices
G = nx.convert_node_labels_to_integers(G, first_label=0)

In [11]:
bet_centrality    = dict(nx.betweenness_centrality(G))
degree_centrality = dict(nx.degree_centrality(G))

In [24]:
pearson_corr, _ = pearsonr(list(bet_centrality.values()), list(degree_centrality.values()))
print("Pearson Correlation:", "%3.4f"%pearson_corr)

Pearson Correlation: 0.8242


## 3) Calcule o coeficiênte de relação de Spearman entre as medidas de Closeness Centrality e K-Core para a rede Jazz

In [25]:
# Read and prepare the network
G = nx.read_edgelist('data/jazz.txt', nodetype=int)
# Grafo não direcionado
G = G.to_undirected()
# Remover auto-loops
G.remove_edges_from(nx.selfloop_edges(G))
# Escolhe maior componente
Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
G = G.subgraph(Gcc[0])
# Renomeia os vértices
G = nx.convert_node_labels_to_integers(G, first_label=0)

In [26]:
close_centrality  = dict(nx.closeness_centrality(G))
k_core = dict(nx.core_number(G))

In [29]:
spearmanr_corr, _ = spearmanr(list(close_centrality.values()), list(k_core.values()))
print("Spearman Correlation:", "%3.4f"%spearmanr_corr)

Spearman Correlation: 0.7353


## 4) Medidas de Centralidade

- **Betweness Centrality**: Menores caminhos de modo a determinar a carga (em termos de quantidade de tráfego) em cada vértice
- **Eigen Vector Centrality**: Calculada a partir do espectro da matriz de adjacências
- **Grau**: É uma medida de centralidade local, pois alguns vértices apresentando valores altos dessa medida podem estar nas bordas da rede
- **PageRang**: Um vértice é considerado central se estiver conectado a outros vértices centrais
- **Random Walk Acessibility**: Calculado a partir da exponencial da matriz de probabilidade de transição
